In [ ]:
import pandas as pd
import datetime
%matplotlib inline

# create train and test dataset
def create():
    new_rows = []
    with open('electricity.txt','r') as f:
        for idx,line in enumerate(f.readlines()):
            if idx >= 1  and idx <= 105215:
                continue
            line = line.split(';')
            line[-1] = line[-1][:-1]
            new_rows.append(line)
    new_rows[0][0] = 'time'
    for idx1, line in enumerate(new_rows[1:]):
        for idx2, consume in enumerate(line[1:]):
            consume = consume.split(',')
            if len(consume) == 1:
                line[idx2 + 1] = float(consume[0])
            elif len(consume) == 2:
                line[idx2 + 1] = float(f'{consume[0]}.{consume[1]}')
        new_rows[idx1 + 1] = line
    data = pd.DataFrame(new_rows[1:], columns=new_rows[0])
    data  = data.set_index('time')
    train = data[0: 128639-105215]
    test = data[128639-105215:-1]
    train.to_csv('train.csv'), test.to_csv('test.csv')
create()

In [37]:
# 15 minute to 1 hour
def min2hour():
    train = pd.read_csv('train.csv').set_index('time')
    test = pd.read_csv('test.csv').set_index('time')
    column = train.columns.to_list()
    column.extend(['time', 'hour', 'dayofweek','age'])
    hour_train = pd.DataFrame(columns=column)
    for idx in range(0, len(train), 4):
        str_time = datetime.datetime.strptime(train.index.values[idx][1:-1],
                                              "%Y-%m-%d %H:%M:%S")
        hour_data = train.iloc[idx:idx + 4].sum(axis=0)
        hour_data['time'] = str_time
        hour_data['hour'] = str_time.hour
        hour_data['dayofweek'] = str_time.weekday()
        hour_data['age'] = idx / 4
        hour_train = hour_train._append(hour_data, ignore_index=True)
    hour_train.to_csv('hour_train.csv', index=False)

    hour_test = pd.DataFrame(columns=column)
    for idx in range(0, len(test), 4):
        str_time = datetime.datetime.strptime(test.index.values[idx][1:-1],
                                              "%Y-%m-%d %H:%M:%S")
        hour_data = test.iloc[idx:idx + 4].sum(axis=0)
        hour_data['time'] = str_time
        hour_data['hour'] = str_time.hour
        hour_data['dayofweek'] = str_time.weekday()
        hour_data['age'] = idx  / 4
        hour_test = hour_test._append(hour_data, ignore_index=True)
    hour_test.to_csv('hour_test.csv', index=False)
min2hour()

In [38]:
# z-score
# def z_score():
cols = ['hour', 'dayofweek', 'age']
train = pd.read_csv('hour_train.csv').set_index('time')
test = pd.read_csv('hour_test.csv').set_index('time')
train[cols] = train[cols].apply(lambda x: (x - x.mean()) / x.std())
test[cols] = test[cols].apply(lambda x: (x - x.mean()) / x.std())

train.fillna(0), test.fillna(0)
train.to_csv('hour_train.csv'), test.to_csv('hour_test.csv')

(None, None)

In [39]:
pd.read_csv('hour_train.csv').set_index('time')

,"""MT_001""","""MT_002""","""MT_003""","""MT_004""","""MT_005""","""MT_006""","""MT_007""","""MT_008""","""MT_009""","""MT_010""",...,"""MT_364""","""MT_365""","""MT_366""","""MT_367""","""MT_368""","""MT_369""","""MT_370""",hour,dayofweek,age
time,,,,,,,,,,,,,,,,,,,,,
2014-01-01 00:00:00,10.152284,96.017070,1.737619,595.528455,302.439024,1104.166667,26.568683,989.898990,204.545455,266.666667,...,4045.454545,69.100391,37.448800,949.956102,185.308848,2898.826979,29405.405405,-1.661183,-0.504060,-1.731607
2014-01-01 01:00:00,11.421320,93.172119,0.000000,583.333333,284.146341,934.523810,23.742227,1000.000000,208.041958,277.419355,...,3636.363636,67.796610,45.640726,911.325724,193.656093,2723.607038,28108.108108,-1.516732,-0.504060,-1.731016
2014-01-01 02:00:00,11.421320,98.150782,0.000000,569.105691,259.756098,851.190476,22.046354,1006.734007,211.538462,249.462366,...,3522.727273,71.707953,40.959626,798.946444,202.003339,2657.624633,26756.756757,-1.372282,-0.504060,-1.730424
2014-01-01 03:00:00,11.421320,87.482219,0.000000,508.130081,258.536585,717.261905,23.742227,831.649832,187.062937,226.881720,...,3431.818182,67.796610,40.959626,734.855136,185.308848,2802.785924,25405.405405,-1.227831,-0.504060,-1.729833
2014-01-01 04:00:00,10.152284,88.904694,0.000000,445.121951,230.487805,625.000000,19.785189,764.309764,167.832168,208.602151,...,3454.545455,70.404172,32.182563,744.512730,190.317195,2714.076246,27351.351351,-1.083380,-0.504060,-1.729241
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2014-09-01 19:00:00,38.071066,168.563300,6.950478,642.276423,226.829268,788.690476,53.702657,1279.461279,283.216783,172.043011,...,27363.636364,419.817471,131.655939,2828.797191,826.377295,4636.363636,106216.216216,1.083380,-1.503983,1.729241
2014-09-01 20:00:00,11.421320,176.386913,6.950478,752.032520,273.170732,1002.976190,54.833239,1259.259259,284.965035,205.376344,...,29113.636364,404.172099,151.550614,2811.237928,734.557596,4361.436950,94540.540541,1.227831,-1.503983,1.729833
2014-09-01 21:00:00,19.035533,150.782361,6.950478,689.024390,290.243902,1068.452381,56.529112,1195.286195,260.489510,267.741935,...,31954.545455,397.653194,119.368051,2453.028973,701.168614,4192.815249,100756.756757,1.372282,-1.503983,1.730424
